<a href="https://colab.research.google.com/github/lehuong240823/web-scraping-toolkit/blob/main/FormulaOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import warnings
import time
from bs4 import BeautifulSoup
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}
make_directory(slash.join([r'/content/drive/MyDrive', 'VR']))
for attempt in range(1):
  warnings.filterwarnings('ignore', message='Unverified HTTPS request')
  session = requests.Session()
  session.verify = False
  try:

    index = 0
    prefix = ''
    for p in range(0, 4):
      if p != 0:
        prefix = prefix + '0'
      for index in range(0, 150):
        image_name = prefix + str(index)
        img_url = 'https://qrguiding.com/upload/c16a5320fa475530d9583c34fd356ef5/files/' + image_name +'.jpg'
        image_filename = slash.join(['/content/drive/MyDrive', 'VR', str(1) + '.jpg'])

        img_page = requests.get(img_url, headers=headers, verify=False)
        #content = img_page.content.decode('utf-8', 'replace')
        #img_soup = bs4.BeautifulSoup(content, 'html.parser')
        #img_element = img_soup.find('img')
        #if img_soup.title is None :
        response = session.get(img_url)
        response.raise_for_status()
        with open(image_filename, 'wb') as f:
          f.write(response.content)
        print(img_url)
        break
  except requests.exceptions.ConnectionError as e:
      print(f"Connection error occurred: {e}")
  except requests.exceptions.RequestException as e:
      print(f"Error downloading image: {e}")
      if attempt < 4:
        print("Retrying...")
        time.sleep(2)



https://qrguiding.com/upload/c16a5320fa475530d9583c34fd356ef5/files/1.jpg
https://qrguiding.com/upload/c16a5320fa475530d9583c34fd356ef5/files/02.jpg
https://qrguiding.com/upload/c16a5320fa475530d9583c34fd356ef5/files/002.jpg


In [ ]:
import traceback
import os
from google.colab import files
import csv
import bs4
import requests
import shutil
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
drive_path = r'/content/drive/MyDrive/F1'

url = r'https://www.formula1.com'
path = r'/content'
slash  = '/'
year_start = 2024
year_end = 2025

drivers_key = 'drivers'
teams_key = 'teams'
schedule_key = 'schedule'
circuit_key = 'circuits'
results_key = 'results'
fastest_laps_key = 'fastest-laps'
flag_key = 'flags'

drivers_path = slash.join([drive_path, drivers_key])
position_path = slash.join([drive_path, drivers_key, 'position'])
profile_path = slash.join([drive_path, drivers_key, 'profile'])
schedule_path = slash.join([drive_path, schedule_key])
teams_path = slash.join([drive_path, teams_key])
cars_path = slash.join([drive_path, teams_key, 'car'])
small_logo_path = slash.join([drive_path, teams_key, 'small logo'])
full_logo_path = slash.join([drive_path, teams_key, 'full logo'])
circuits_path = slash.join([drive_path, circuit_key])
results_path = slash.join([drive_path, results_key])

drivers_rel_path = '../img/drivers'
teams_rel_path = '../img/teams'

pos_rel_path = slash.join([drivers_rel_path, 'position'])
profile_rel_path = slash.join([drivers_rel_path, 'profile'])
cars_rel_path = slash.join([teams_rel_path, 'car'])
small_logo_rel_path = slash.join([teams_rel_path, 'small logo'])
full_logo_rel_path = slash.join([teams_rel_path, 'full logo'])


drivers_csv = slash.join([drivers_path, drivers_key + '.csv'])
teams_csv = slash.join([teams_path, teams_key + '.csv'])
schedule_csv = slash.join([schedule_path, schedule_key + '.csv'])
sessions_csv = slash.join([schedule_path, 'sessions' + '.csv'])
circuits_csv = slash.join([circuits_path, circuit_key + '.csv'])
results_csv = slash.join([results_path, results_key + '.csv'])
fastest_lap_csv = slash.join([results_path, fastest_laps_key + '.csv'])

drivers_field = ['driver_id', 'driver_name', 'team_short_name', 'country', 'podiums', 'points', 'grands_prix_entered', 'world_championships', 'highest_race_finish', 'highest_grid_position', 'date_of_birth', 'place_of_birth', 'position', 'position_png', 'transparent_profile_png', 'profile_avif']
teams_field = ['team_id', 'team_short_name', 'full_name', 'base', 'team_chief', 'technical_chief', 'chassis', 'power_unit', 'first_team_entry', 'world_championships', 'highest_race_finish', 'pole_positions', 'fastest_laps', 'car_png', 'small_logo_png', 'full_logo_avif']
schedule_field = ['schedule_id', 'year', 'month', 'date', 'round', 'grand_prix', 'grand_prix_title']
sessions_field = ['session_id', 'schedule_id', 'month', 'date', 'name']
circuits_field = ['circuit_id', 'schedule_id', 'grand_prix', 'name', 'first_grand_prix', 'number_of_laps', 'circuit_length', 'race_distance', 'lap_record', 'carbon_png', 'circuit_avif']
results_field = ['results_id', 'type', 'year', 'grand_prix', 'pos', 'no', 'driver', 'car', 'laps', 'time_retired', 'pts']
fastest_lap_field = ['fastest_lap_id', 'year', 'grand_prix', 'pos', 'no', 'driver', 'car', 'laps', 'time_of_day', 'time', 'avg_speed']

def make_directory(dirname):
  if os.path.exists(dirname):
    for filename in os.listdir(dirname):
          file_path = os.path.join(dirname, filename)
          if os.path.isfile(file_path):
              os.remove(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
  else:
    os.makedirs(dirname)

def download_img(img_url, img_path):
  img_data = requests.get(img_url).content
  with open(img_path, 'wb') as handler:
    handler.write(img_data)

def download_folder(dir_to_zip, zip_name):
  output_filename = zip_name + '.zip'
  delete_dir_after_download = "No"
  os.system("zip -r {} {}".format(output_filename , dir_to_zip ) )
  if delete_dir_after_download == "Yes":
    os.system("rm -r {}".format(dir_to_zip ) )
  files.download(output_filename)

Mounted at /content/drive


# **DRIVERS**

In [ ]:
def download_png():
  pos_path = slash.join([position_path, pos_file_name])
  png_path = slash.join([profile_path, png_file_name])

  download_img(drivers_pos['src'], pos_path)
  download_img(drivers_png['src'], png_path)
try:
  rel_path = '../img/drivers/'
  make_directory(drivers_path)
  make_directory(profile_path)
  make_directory(position_path)
  drivers_page = requests.get(slash.join([url, 'en', drivers_key]))
  drivers_soup = bs4.BeautifulSoup(drivers_page.content, 'html.parser')

  a_elements = drivers_soup.find_all('a', class_="group focus-visible:outline-0")

  with open(drivers_csv, mode='w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=drivers_field)
    writer.writeheader()
    did = 1
    for element in a_elements:
      driver_url = url + element['href']
      driver_page = requests.get(driver_url)
      driver_soup = bs4.BeautifulSoup(driver_page.content, 'html.parser')

      full_name = element.find('div', class_='flex gap-xxs flex-col f1-driver-name')
      driver_name = ' '.join([p.text for p in full_name.find_all('p')])
      drivers_pos = full_name.find_next('img', class_="f1-c-image f1-utils-square-block text-[6rem]")
      drivers_png = drivers_pos.find_next('img', class_="f1-c-image ml-0 mr-0 pr-s max-w-3/4")
      name_element = driver_soup.find('h1', class_='f1-heading tracking-normal text-fs-24px tablet:text-fs-42px leading-tight normal-case font-bold non-italic f1-heading__body font-formulaOne')

      if(name_element is not None):
        driver_infor = name_element.find_all_next('dd', class_='f1-text font-titillium tracking-normal font-normal non-italic normal-case leading-snug f1-text__body text-fs-17px max-laptop:mb-normal')
        driver_avif_img = name_element.find_previous('img', class_='f1-c-image aspect-square laptop:aspect-4/3 desktop:aspect-16/10 w-full overflow-hidden object-cover object-top')
        driver_infor = [infor.text for infor in driver_infor]
        avif_file_name = (driver_name + '.avif').replace(' ', '-').lower()
        avif_path = profile_path + '/' + avif_file_name
        if driver_avif_img is not None:
          download_img(driver_avif_img['src'], avif_path)
      else:
        team = element.find('p', class_='f1-heading tracking-normal text-fs-12px leading-tight normal-case font-normal non-italic f1-heading__body font-formulaOne text-greyDark')
        driver_infor = [team.text, None, None, None, None, None, None, None, None, None]
        avif_file_name = None

      pos_file_name = (driver_name + ' position' + '.png').replace(' ', '-').lower()
      png_file_name = (driver_name + '.png').replace(' ', '-').lower()

      download_png()
      if avif_file_name is not None:
        avif_file_name = slash.join([profile_rel_path, avif_file_name])

      driver_profile = [did, driver_name]  + driver_infor + [drivers_pos['alt'], slash.join([pos_rel_path, pos_file_name]), slash.join([profile_rel_path, png_file_name]), avif_file_name]

      writer.writerow(dict(zip(drivers_field, driver_profile)))
      did += 1
except Exception:
  print(traceback.format_exc())

# **TEAMS**

In [ ]:
try:
  make_directory(teams_path)
  make_directory(cars_path)
  make_directory(small_logo_path)
  make_directory(full_logo_path)
  with open(teams_csv, mode='w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=teams_field)
    writer.writeheader()
    tid = 1
    teams_page = requests.get(slash.join([url, 'en', teams_key]))
    teams_soup = bs4.BeautifulSoup(teams_page.content, 'html.parser')
    a_elements = teams_soup.find_all('a', class_="group focus-visible:outline-0")
    for element in a_elements:
      team_url = slash.join([url, element['href']])
      team_page = requests.get(team_url)
      team_soup = bs4.BeautifulSoup(team_page.content, 'html.parser')
      team_sname = element.find('span', class_='f1-heading tracking-normal text-fs-20px tablet:text-fs-25px leading-tight normal-case font-bold non-italic f1-heading__body font-formulaOne').text
      sname_element = team_soup.find('h1', class_='f1-heading tracking-normal text-fs-24px tablet:text-fs-42px leading-tight normal-case font-normal non-italic f1-heading__body font-formulaOne')
      if(sname_element is not None):
        full_logo = team_soup.find('img', class_='f1-c-image w-full h-full object-cover')
        full_logo_avif =  (team_sname + ' full logo' + '.avif').replace(' ', '-').lower()
        download_img(full_logo['src'], slash.join([drive_path, teams_key, 'full logo', full_logo_avif]))
        team_infor = team_soup.find_all('dd', class_='f1-text font-titillium tracking-normal font-normal non-italic normal-case leading-snug f1-text__body text-fs-17px max-laptop:mb-normal')
      else:
        team_infor = []

      small_logo = element.find('img', class_="f1-c-image h-[2em] ml-auto mr-0")
      small_logo_png = (team_sname + ' small logo' + '.png').replace(' ', '-').lower()
      download_img(small_logo['src'], slash.join([drive_path, teams_key, 'small logo', small_logo_png]))

      car = element.find_all('img', class_="f1-c-image")[3]
      car_png = (team_sname + ' car' + '.png').replace(' ', '-').lower()
      download_img(car['src'], slash.join([cars_path, car_png]))

      team_profile = [tid, team_sname] + [infor.text for infor in team_infor] + [slash.join([cars_rel_path, car_png]), slash.join([small_logo_rel_path, small_logo_png]), slash.join([full_logo_rel_path, full_logo_avif])]
      writer.writerow(dict(zip(teams_field, team_profile)))
      tid +=1
except Exception:
    print(traceback.format_exc())


# **UPDATE DRIVERS**

In [ ]:
drivers_df = pd.read_csv(drivers_csv)
teams_df = pd.read_csv(teams_csv)
update_drivers = pd.merge(drivers_df, teams_df, on ='team_short_name', how ='left')
update_drivers.rename(columns = {'world_championships_x': 'world_championships'}, inplace = True)
update_drivers.rename(columns = {'highest_race_finish_x': 'highest_race_finish'}, inplace = True)
drivers_df = update_drivers[drivers_df.columns]
drivers_df['team_short_name'] = update_drivers['team_id']
drivers_df.rename(columns = {'team_short_name': 'team_id'}, inplace = True)
drivers_df.to_csv(drivers_csv, index=False)

<ipython-input-14-bfd36d1a2303>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drivers_df['team_short_name'] = update_drivers['team_id']
<ipython-input-14-bfd36d1a2303>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drivers_df.rename(columns = {'team_short_name': 'team_id'}, inplace = True)


# **SCHEDULE**

In [ ]:
try:
  make_directory(schedule_path)
  with open(schedule_csv,  mode='w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=schedule_field)
    writer.writeheader()
    rid = 1
    for year in range(year_start, year_end):
      schedule_page = requests.get(url + '/en/' + 'racing' + '/' + str(year))
      schedule_soup = bs4.BeautifulSoup(schedule_page.content, 'html.parser')
      round_elements = schedule_soup.find_all('a', class_ = 'outline-offset-4 outline-scienceBlue group outline-0 focus-visible:outline-2')
      for round_element in round_elements:
        round_url = url + round_element['href']
        #round_page = requests.get(round_url)
        #round_soup = bs4.BeautifulSoup(round_page.content, 'html.parser')
        round_name = round_element.find('p', class_ = 'f1-text font-titillium tracking-normal font-bold non-italic uppercase leading-snug f1-text__micro text-fs-15px text-brand-primary').text
        date_elements = round_element.findAll('span', class_ = 'whitespace-nowrap')
        for date_element in date_elements:
          if date_element.text[0].isnumeric():
            round_date = date_element.text
          else:
            round_month = date_element.text
        if round_name == 'FESTIVAL':
          round_country = round_element.find('p', class_ = 'f1-heading tracking-normal text-fs-14px leading-tight normal-case font-normal non-italic f1-heading__body font-formulaOne text-brand-white')
        else:
          round_country = round_element.find('p', class_ = 'f1-heading tracking-normal text-fs-18px leading-tight normal-case font-bold non-italic f1-heading__body font-formulaOne overflow-hidden')
        if round_country is not None:
          round_country = round_country.text

        if round_name == 'FESTIVAL':
          round_grand_prix = round_element.find('p', class_ = 'f1-heading tracking-normal text-fs-18px leading-tight normal-case font-bold non-italic f1-heading__body font-formulaOne text-brand-white')
        else:
          round_grand_prix = round_element.find('p', class_ = 'f1-heading tracking-normal text-fs-12px leading-tight normal-case font-normal non-italic f1-heading__body font-formulaOne')
        if round_grand_prix is not None:
          round_grand_prix = round_grand_prix.text
        round_data = [rid, year, round_month, round_date, round_name, round_country, round_grand_prix]
        writer.writerow(dict(zip(schedule_field, round_data)))
        rid += 1
except Exception:
    print(traceback.format_exc())



# **SESSIONS**

In [ ]:
try:
  with open(sessions_csv,  mode='w', encoding='utf-8', newline='') as csvfile:

    writer = csv.DictWriter(csvfile, fieldnames=sessions_field)
    writer.writeheader()
    sid = 1
    seid = 1
    for year in range(year_start, year_end):
      schedule_page = requests.get(url + '/en/' + 'racing' + '/' + str(year))
      schedule_soup = bs4.BeautifulSoup(schedule_page.content, 'html.parser')
      round_elements = schedule_soup.find_all('a', class_ = 'outline-offset-4 outline-scienceBlue group outline-0 focus-visible:outline-2')
      for round_element in round_elements:
        round_url = url + round_element['href']
        round_page = requests.get(round_url)
        round_soup = bs4.BeautifulSoup(round_page.content, 'html.parser')
        session_elements = round_soup.find_all('div', class_ = 'relative px-xs py-s tablet:p-normal tablet:pl-0 tablet:pr-normal rounded-md flex flex-wrap tablet:flex-nowrap mt-micro items-center bg-white pr-l')
        for session_element in reversed(session_elements):
          session_date = session_element.find('p', class_ = 'f1-heading tracking-normal text-fs-18px leading-none normal-case font-normal non-italic f1-heading__body font-formulaOne').text
          session_month = session_element.find('div', class_ = 'rounded-xl py-0.5 px-2 mt-1 leading-none inline-block bg-lightGray text-grey-70').text
          session_name = session_element.find('span', class_ = 'f1-heading tracking-normal text-fs-18px leading-tight normal-case font-bold non-italic f1-heading__body font-formulaOne block mb-xxs').text
          session_data = [seid, sid, session_month, session_date, session_name]
          writer.writerow(dict(zip(sessions_field, session_data)))
          seid += 1
        sid += 1
except Exception:
    print(traceback.format_exc())


# **RESULTS**

In [ ]:
try:
  make_directory(results_path)
  with open(results_csv,  mode='w', encoding='utf-8', newline='') as csvfile:

    writer = csv.DictWriter(csvfile, fieldnames=results_field)
    writer.writeheader()

    for year in range(year_start, year_end):
      results_page = requests.get(url + '/en/' + results_key + '/' + str(year) + '/races')
      results_soup = bs4.BeautifulSoup(results_page.content, 'html.parser')
      a_elements = results_soup.find_all('a', class_ = 'block')
      rid = 1
      for a_element in a_elements:
        if a_element.text == 'All':
          all_grand_prix_elements = a_element
          for next_sibling in all_grand_prix_elements.parent.next_siblings:
            grand_prix_url = url + next_sibling.findChildren('a', class_ = 'block')[0]['href']
            grand_prix_page = requests.get(grand_prix_url)
            grand_prix_soup = bs4.BeautifulSoup(grand_prix_page.content, 'html.parser')
            race_result_table = grand_prix_soup.find_all('table', class_ = 'f1-table f1-table-with-data w-full')
            if len(race_result_table) > 0 and race_result_table[0] is not None:
              race_result_table = race_result_table[0]
              table_rows = race_result_table.find('tbody').find_all('tr')
              for row in table_rows:
                result_data = [rid, 'race', year, next_sibling.text]
                cells = row.findChildren('td')
                for cell in cells:
                  if len(cell.findChildren('span'))==3:
                    result_data.append(cell.findChildren('span')[0].text + ' ' + cell.findChildren('span')[1].text)
                  else:
                    result_data.append(cell.text)
                writer.writerow(dict(zip(results_field, result_data)))
                rid += 1

            result_a_element = grand_prix_soup.find_all('a', class_ = 'block')
            for a_element in result_a_element:
              if a_element.text == 'Sprint':
                sprint_url = url + a_element['href']
                sprint_page = requests.get(sprint_url)
                sprint_soup = bs4.BeautifulSoup(sprint_page.content, 'html.parser')
                sprint_table = sprint_soup.find_all('table', class_ = 'f1-table f1-table-with-data w-full')
                if len(sprint_table) > 0 and sprint_table[0] is not None:
                  sprint_table = sprint_table[0]
                  table_rows = sprint_table.find('tbody').find_all('tr')
                  for row in table_rows:
                    result_data = [rid, 'sprint', year, next_sibling.text]
                    cells = row.findChildren('td')
                    for cell in cells:
                      if len(cell.findChildren('span'))==3:
                        result_data.append(cell.findChildren('span')[0].text + ' ' + cell.findChildren('span')[1].text)
                      else:
                        result_data.append(cell.text)
                    writer.writerow(dict(zip(results_field, result_data)))
                    rid +=1
          break
except Exception:
    print(traceback.format_exc())



# **UPDATE RESULT**



In [ ]:
results_df = pd.read_csv(fastest_lap_csv)
schedule_df = pd.read_csv(schedule_csv)
update_results = pd.merge(results_df, schedule_df, on =['year', 'grand_prix'], how ='left')
results_df = update_results[results_df.columns]
results_df['year'] = update_results['schedule_id']
results_df.rename(columns = {'year': 'schedule_id'}, inplace = True)
results_df = results_df.drop(columns=['grand_prix'])
results_df.to_csv(fastest_lap_csv, index=False)



<ipython-input-16-a18f70647285>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['year'] = update_results['schedule_id']
<ipython-input-16-a18f70647285>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df.rename(columns = {'year': 'schedule_id'}, inplace = True)


In [ ]:
results_df = pd.read_csv(fastest_lap_csv)
driver_df = pd.read_csv(drivers_csv)
driver_df.rename(columns = {'driver_name': 'driver'}, inplace = True)
update_results = pd.merge(results_df, driver_df, on =['driver'], how ='left')

a = str(results_df.loc[0, 'driver'])
b = str(driver_df.loc[0, 'driver'])

results_df = update_results[results_df.columns]
results_df['driver'] = update_results['driver_id']
results_df.rename(columns = {'driver': 'driver_id'}, inplace = True)
results_df = results_df.drop(columns=['pos'])
results_df = results_df.drop(columns=['no'])

results_df.to_csv(fastest_lap_csv, index=False)


<ipython-input-17-93eea7ddaeb8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['driver'] = update_results['driver_id']
<ipython-input-17-93eea7ddaeb8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df.rename(columns = {'driver': 'driver_id'}, inplace = True)


# **FASTEST LAP**

In [ ]:
try:
  with open(fastest_lap_csv,  mode='w', encoding='utf-8', newline='') as csvfile:

    writer = csv.DictWriter(csvfile, fieldnames=fastest_lap_field)
    writer.writeheader()

    for year in range(year_start, year_end):
      results_page = requests.get(url + '/en/' + results_key + '/' + str(year) + '/races')
      results_soup = bs4.BeautifulSoup(results_page.content, 'html.parser')
      a_elements = results_soup.find_all('a', class_ = 'block')
      rid = 1
      for a_element in a_elements:
        if a_element.text == 'All':
          all_round_elements = a_element
          for next_sibling in all_round_elements.parent.next_siblings:
            round_url = url + next_sibling.findChildren('a', class_ = 'block')[0]['href']
            round_url = round_url.replace('race-result', 'fastest-laps')
            round_page = requests.get(round_url)
            round_soup = bs4.BeautifulSoup(round_page.content, 'html.parser')
            race_result_table = round_soup.find_all('table', class_ = 'f1-table f1-table-with-data w-full')
            if len(race_result_table) > 0 and race_result_table[0] is not None:
              race_result_table = race_result_table[0]
              table_rows = race_result_table.find('tbody').find_all('tr')
              for row in table_rows:
                result_data = [rid, year, next_sibling.text]
                cells = row.findChildren('td')
                for cell in cells:
                  if len(cell.findChildren('span'))==3:
                    result_data.append(cell.findChildren('span')[0].text + ' ' + cell.findChildren('span')[1].text)
                  else:
                    result_data.append(cell.text)
                writer.writerow(dict(zip(fastest_lap_field, result_data)))
                rid += 1
          break
except Exception:
    print(traceback.format_exc())

# **UPDATE FLAPS**

In [ ]:
fastest_laps_df = pd.read_csv(fastest_lap_csv)
schedule_df = pd.read_csv(schedule_csv)
update_results = pd.merge(fastest_laps_df, schedule_df, on =['year', 'grand_prix'], how ='left')
fastest_laps_df = update_results[fastest_laps_df.columns]
fastest_laps_df['year'] = update_results['schedule_id']
fastest_laps_df.rename(columns = {'year': 'schedule_id'}, inplace = True)
fastest_laps_df = fastest_laps_df.drop(columns=['grand_prix'])
fastest_laps_df.to_csv(fastest_lap_csv, index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/F1/results/fastest-laps.csv'

# **CIRCUITS**

In [ ]:
import requests
import chardet
from bs4 import BeautifulSoup

# Fetch the page
url = url + '/en/' + 'racing' + '/' + str(2024)
response = requests.get(url)

# Detect encoding
result = chardet.detect(response.content)
encoding = result['encoding']

# Set the encoding based on detection
response.encoding = encoding
print(response.encoding)
# Parse the content with Beautiful Soup
soup = BeautifulSoup(response.text, 'html.parser')


utf-8


In [ ]:
try:
  rel_path = '../img/circuits/'
  make_directory(circuits_path)
  with open(circuits_csv,  mode='w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=circuits_field)
    writer.writeheader()
    sid = 1
    cid = 1
    for year in range(year_start, year_end):
      schedule_page = requests.get(url + '/en/' + 'racing' + '/' + str(year))
      schedule_soup = bs4.BeautifulSoup(schedule_page.content, 'html.parser')
      round_elements = schedule_soup.find_all('a', class_ = 'outline-offset-4 outline-scienceBlue group outline-0 focus-visible:outline-2')
      for round_element in round_elements:
        round_url = url + round_element['href']
        round_page = requests.get(round_url)
        round_soup = bs4.BeautifulSoup(round_page.content, 'html.parser')
        round_name = round_element.find('p', class_ = 'f1-text font-titillium tracking-normal font-bold non-italic uppercase leading-snug f1-text__micro text-fs-15px text-brand-primary').text
        if round_name == 'FESTIVAL':
          round_country = round_element.find('p', class_ = 'f1-heading tracking-normal text-fs-14px leading-tight normal-case font-normal non-italic f1-heading__body font-formulaOne text-brand-white')
        else:
          round_country = round_element.find('p', class_ = 'f1-heading tracking-normal text-fs-18px leading-tight normal-case font-bold non-italic f1-heading__body font-formulaOne overflow-hidden')
        if round_country is not None:
          round_country = round_country.text
        circuit_country = round_country

        circuit_data = [cid, sid, circuit_country]
        circuit_carbon_img = round_soup.find('img', class_ = 'desktop:w-[70%] laptop:w-2/5 w-60 mx-auto')
        if circuit_carbon_img is not None:
          carbon_name = (circuit_country + ' carbon' + '.png').replace(' ', '-').lower()
          #carbon_path = slash.join([circuits_path, carbon_name])
          download_img(circuit_carbon_img['src'], slash.join([circuits_path, carbon_name]))

        circuit_elements = round_soup.find_all('a', class_ = 'grid grid-flow-col auto-cols-max rounded-5 cursor-pointer items-center transition-colors duration-200 font-titillium font-[600] w-full min-w-max laptop:w-auto laptop:text-center laptop:auto-cols-auto focus-visible:outline focus-visible:outline-offset-2 focus-visible:outline-[3px] focus-visible:outline-carbonBlack disabled:pointer-events-none disabled:opacity-75 disabled:cursor-default text-12 px-[15px] gap-[8px] h-[39px] !rounded-none !border-b !border-b-gray20 !auto-cols-auto !text-left !min-w-fit !flex !text-15 !h-auto py-2.5 pl-0 focus-visible:!outline-2 hover:!border-b-primary hover:!text-primary focus:!border-b-primary focus:!text-primary !text-15')
        for circuit_element in circuit_elements:
          if circuit_element is not None and circuit_element['data-navigation-element'] == 'circuitStats':
            circuit_page = requests.get(circuit_element['href'])
            circuit_soup = bs4.BeautifulSoup(circuit_page.content, 'html.parser')
            circuit_img = circuit_soup.find('img', class_ = 'f1-c-image w-full h-auto object-cover')
            if circuit_img is not None:
              circuit_img_name = (circuit_data[2] + ' circuit' + '.avif').replace(' ', '-').lower()
              download_img(circuit_img['src'], slash.join([circuits_path, circuit_img_name]))
            circuit_name = circuit_soup.find('h2', class_ = 'f1-heading tracking-normal text-fs-20px tablet:text-fs-25px leading-none normal-case font-bold non-italic f1-heading__body font-formulaOne block laptop:hidden')
            if circuit_name is not None:
              circuit_name = circuit_name.text
              circuit_infor = circuit_soup.find_all('h2', class_ = 'f1-heading tracking-normal text-fs-22px tablet:text-fs-32px leading-tight normal-case font-bold non-italic f1-heading__body font-formulaOne')

              circuit_data = circuit_data + [circuit_name] + [infor.text for infor in circuit_infor] + [rel_path + carbon_name, rel_path + circuit_img_name]
              #print(circuit_data)
        writer.writerow(dict(zip(circuits_field, circuit_data)))
        cid += 1
        sid += 1
except Exception:
    print(traceback.format_exc())


In [ ]:
circuits_df = pd.read_csv(circuits_csv)
circuits_df.fillna('', inplace=True)
circuits_df.to_csv(circuits_csv, index=False)

In [ ]:
circuits_df = pd.read_csv(circuits_csv)
schedule_df = pd.read_csv(schedule_csv)
update_schedule = pd.merge(schedule_df, circuits_df, on ='schedule_id', how ='left')
schedule_df.insert(loc=schedule_df.columns.get_loc('schedule_id') + 1, column='circuit_id', value=circuits_df['circuit_id'])
schedule_df = schedule_df.drop(columns=['grand_prix'])
schedule_df.to_csv(schedule_csv, index=False)
circuits_df = circuits_df.drop(columns=['schedule_id'])
circuits_df.to_csv(circuits_csv, index=False)

In [ ]:
df = pd.read_csv(circuits_csv)
print(df)

    circuit_id      grand_prix                                          name  \
0            1          Sakhir                 Bahrain International Circuit   
1            2         Bahrain                 Bahrain International Circuit   
2            3    Saudi Arabia                       Jeddah Corniche Circuit   
3            4       Australia                Albert Park Grand Prix Circuit   
4            5           Japan                                Suzuka Circuit   
5            6           China                Shanghai International Circuit   
6            7           Miami                 Miami International Autodrome   
7            8  Emilia-Romagna  Autodromo Internazionale Enzo e Dino Ferrari   
8            9          Monaco                             Circuit de Monaco   
9           10          Canada                     Circuit Gilles-Villeneuve   
10          11           Spain                Circuit de Barcelona-Catalunya   
11          12         Austria          

# **FLAGS**

In [ ]:
try:
  make_directory(drive_path + flag_key)
  make_directory(drive_path + flag_key + drivers_key)
  make_directory(drive_path + flag_key + schedule_key)

  drivers_page = requests.get(url + '/en/' + drivers_key)
  drivers_soup = bs4.BeautifulSoup(drivers_page.content, 'html.parser')
  driver_flags= drivers_soup.find_all('img', class_="f1-c-image h-[2em] ml-auto mr-0 border border-greyDark rounded-xxs")
  for flag in driver_flags:
    flag_name = flag['alt']
    flag_url = flag['src']
    flag_path = drive_path + flag_key + '/' + drivers_key + '/' + flag_name + '.jpg'
    download_img(flag_url, flag_path)


  for year in range(year_start, year_end):
    schedule_page = requests.get(url + '/en/' + 'racing' + '/' + str(year))
    schedule_soup = bs4.BeautifulSoup(schedule_page.content, 'html.parser')
    wrappers_div = schedule_soup.find_all(class_='f1-inner-wrapper flex flex-col gap-xs overflow-clip h-full overflow-visible outline-brand-primary')
    for wrapper in wrappers_div:
      schedule_flag = wrapper.find('img', class_="f1-c-image h-[1.625rem] ml-auto mr-0 rounded-xxs border border-greyDark")
      flag_name = wrapper.find('p', class_="f1-heading tracking-normal text-fs-18px leading-tight normal-case font-bold non-italic f1-heading__body font-formulaOne overflow-hidden")
      if schedule_flag is None:
        schedule_flag = wrapper.find('img', class_="f1-c-image h-[1.125rem] ml-auto mr-0 rounded-xxs border border-greyDark")
      if schedule_flag is not None and flag_name is not None:
        flag_name = flag_name.text
        flag_url = schedule_flag['src']
        flag_path = drive_path + flag_key + '/' + schedule_key + '/' + flag_name + '.png'
        download_img(flag_url, flag_path)

except Exception:
    print(traceback.format_exc())
